# Using Linear model

- for starting point



# 1)-Importing key Modules

In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install dynet

In [0]:
from collections import defaultdict
import math
import time
import random
import dynet as dy
import numpy as np

In [4]:
! pip install version_information

In [5]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,torch,numpy

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
torch 1.3.1+cu100
numpy 1.17.4
Sat Nov 16 16:29:35 2019 UTC

# 2)- Setting up neural network

In [0]:
# The length of the n-gram
N = 2

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
S = w2i["<s>"]
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      yield [w2i[x] for x in line.strip().split(" ")]

# 3)- Loading Data

### loading data using traditional format
using read()

In [0]:
train = list(read_dataset("train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("valid.txt"))
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)

# 4)- Model Building

### 4.1)- Start DyNet and define trainer

In [0]:
model = dy.ParameterCollection()
trainer = dy.SimpleSGDTrainer(model, learning_rate=0.1)

### 4.2)-Define the model

In [0]:
W_sm = [model.add_lookup_parameters((nwords, nwords)) for _ in range(N)] # Word weights at each position
b_sm = model.add_parameters((nwords))                # Softmax bias

### 4.3)-A function to calculate scores for one value

In [0]:
def calc_score_of_history(words):
  # Create a list of things to sum up with only the bias vector at first
  score_vecs = [b_sm]
  for word_id, lookup_param in zip(words, W_sm): 
    score_vecs.append(lookup_param[word_id])
  return dy.esum(score_vecs)

### 4.4)-Calculate the loss value for the entire sentence

In [0]:
def calc_sent_loss(sent):
  # Create a computation graph
  dy.renew_cg()
  # The initial history is equal to end of sentence symbols
  hist = [S] * N
  # Step through the sentence, including the end of sentence token
  all_losses = []
  for next_word in sent + [S]:
    s = calc_score_of_history(hist)
    all_losses.append(dy.pickneglogsoftmax(s, next_word))
    hist = hist[1:] + [next_word]
  return dy.esum(all_losses)

### 4.5)-Generate a sentence

In [0]:
MAX_LEN = 100

In [0]:
def generate_sent():
  dy.renew_cg()
  hist = [S] * N
  sent = []
  while True:
    p = dy.softmax(calc_score_of_history(hist)).npvalue()
    next_word = np.random.choice(nwords, p=p/p.sum())
    if next_word == S or len(sent) == MAX_LEN:
      break
    sent.append(next_word)
    hist = hist[1:] + [next_word]
  return sent

# 5)- Train Model

In [14]:
for ITER in range(10):
  # Perform training
  random.shuffle(train)
  train_words, train_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(train):
    my_loss = calc_sent_loss(sent)
    train_loss += my_loss.value()
    train_words += len(sent)
    my_loss.backward()
    trainer.update()
    if (sent_id+1) % 5000 == 0:
      print("--finished %r sentences" % (sent_id+1))
  print("iter %r: train loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, train_loss/train_words, math.exp(train_loss/train_words), time.time()-start))

--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
--finished 40000 sentences
iter 0: train loss/word=6.2143, ppl=499.8656, time=99.87s
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
--finished 40000 sentences
iter 1: train loss/word=5.7742, ppl=321.8879, time=98.28s
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 sentences
--finished 40000 sentences
iter 2: train loss/word=5.5815, ppl=265.4671, time=102.50s
--finished 5000 sentences
--finished 10000 sentences
--finished 15000 sentences
--finished 20000 sentences
--finished 25000 sentences
--finished 30000 sentences
--finished 35000 se

# 5)- Evaluate

In [15]:
dev_words, dev_loss = 0, 0.0
start = time.time()
for sent_id, sent in enumerate(dev):
  my_loss = calc_sent_loss(sent)
  dev_loss += my_loss.value()
  dev_words += len(sent)
print("iter %r: dev loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, dev_loss/dev_words, math.exp(dev_loss/dev_words), time.time()-start))

iter 9: dev loss/word=5.4608, ppl=235.2843, time=3.49s


# 6)-Generate a few sentences

In [16]:
for _ in range(5):
  sent = generate_sent()
  print(" ".join([i2w[x] for x in sent]))

<unk> worried that the bugs wo n't measure
the return came to spreads $ N billion to N barrels at a $ N billion yen
for example $ N a share a year earlier the president to <unk> eastern exhibition underestimated <unk> culmination on <unk> relations down or in N futures of N N N N from $ N million of existing <unk> in eastern europe
indeed the telephone lines of the american <unk> & inner-city eventual ciba-geigy turner by the fed are not only by some spring producing free low-cost on <unk> an analyst with <unk> the army company firm legislative force as an clues spill <unk> young said
the average where none <unk> <unk> up gives important before the house energy out should be spent again on nov. N and increase fraud holders
